# todo
More quiality data:
* take games with different openings to ensure variaty - maybe allow moer openings?
* train only on roughly fair, equal possitions

* sample best move from legal predicted ones not just take the best!!!

* make mark3.3 play mark3

* Java App for visulization

* Re-enforcement Learning


In [1]:
import os
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.optim.lr_scheduler import OneCycleLR # type: ignore
import math
from torch.utils.data import DataLoader # type: ignore
from chess import pgn # type: ignore
import tqdm # type: ignore
from dataset import ChessDataset
from model import ChessModel
from helper_funcs import collect_unique_moves, create_input_for_nn
from helper_funcs import process_data_and_save_chunks
import pickle
import gc

# Data Processing

## load data - into chunks so that memory is not overwhelmed, store them in sepearte folder

In [7]:
files = [os.path.join("../data/pgn", file) for file in os.listdir("../data/pgn") if file.endswith(".pgn")]
files.sort()  # Ensure consistent order
LIMIT_OF_FILES = min(len(files), 28)
files = files[:LIMIT_OF_FILES]

max_games = 300000
chunk_size = 10000

# Collect unique moves
move_to_int, num_classes = collect_unique_moves(files, max_games=max_games)

with open("../models/mark3_move_to_int.pkl", "wb") as file:
    pickle.dump(move_to_int, file)
    
print("Number of classes: ", num_classes)

In [3]:
data_chunk_files = process_data_and_save_chunks(files, move_to_int, chunk_size=chunk_size, max_games=max_games)

Processing Data Chunks:   0%|          | 0/2 [16:40<?, ?it/s]


## setup

In [4]:
# Check for GPU
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using MPS backend on Apple Silicon (M2).")
else:
    device = torch.device('cpu')
    print("MPS backend not available. Using CPU.")
    
# Model Initialization
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()

Using MPS backend on Apple Silicon (M2).


# Train

In [5]:
num_epochs = 25

batch_size = 64  # Ensure this matches your DataLoader batch_size
total_batches_per_epoch = 0
for data_chunk_file in data_chunk_files:
    data = np.load(data_chunk_file)
    num_samples = data['X'].shape[0]
    num_batches = math.ceil(num_samples / batch_size)
    total_batches_per_epoch += num_batches

total_steps = num_epochs * total_batches_per_epoch

optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = OneCycleLR(optimizer, max_lr=0.001, total_steps=total_steps)

# Training loop
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    total_batches = 0
    for data_chunk_file in tqdm.tqdm(data_chunk_files, desc=f'Epoch {epoch+1}/{num_epochs}'):
        # Load data chunk
        data = np.load(data_chunk_file)
        X = torch.tensor(data['X'], dtype=torch.float32)
        y = torch.tensor(data['y'], dtype=torch.long)
        # Create Dataset and DataLoader
        dataset = ChessDataset(X, y)
        dataloader = DataLoader(dataset, batch_size=64, num_workers=4, shuffle=True)
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
            optimizer.zero_grad()
            outputs = model(inputs)  # Raw logits
            # Compute loss
            loss = criterion(outputs, labels)
            loss.backward()
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()  # Update learning rate
            running_loss += loss.item()
            total_batches += 1
        # Free up memory
        del X, y, dataset, dataloader
        gc.collect()

    end_time = time.time()
    epoch_time = end_time - start_time
    minutes = int(epoch_time // 60)
    seconds = int(epoch_time % 60)
    avg_loss = running_loss / total_batches
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, Time: {minutes}m{seconds}s')

Epoch 1/25: 100%|██████████| 30/30 [23:53<00:00, 47.78s/it]


Epoch 1/25, Loss: 3.1254, Time: 23m53s


Epoch 2/25: 100%|██████████| 30/30 [24:04<00:00, 48.15s/it]


Epoch 2/25, Loss: 2.4327, Time: 24m4s


Epoch 3/25: 100%|██████████| 30/30 [24:13<00:00, 48.44s/it]


...

Epoch 14/25, Loss: 1.9427, Time: 25m32s


Epoch 15/25: 100%|██████████| 30/30 [25:27<00:00, 50.92s/it]


Epoch 15/25, Loss: 1.9259, Time: 25m27s


Epoch 16/25:  10%|█         | 3/30 [02:32<22:55, 50.94s/it]


EOFError: No data left in file

In [6]:
# Save the model
torch.save(model.state_dict(), "../models/mark3-15e-300k.pth")

# Mark 1 - 10e - 50k
* base; 5k chunks, 14 boards reprezentation
* 10 minutes per epoch - 50k games

# Mark 2 - 20e - 100k
* 10k chunks
* adaptive learning rate
* probabilistic favourizm of mid-late game states
* batch normalization after convolutional layer

# Mark 3
* 300k - 10k chunks - 25e
* its useable :D - forgot to load the mapping and data is sampled randomly
* added additional garbage collection at the ends of the epochs
* better to go for moer games 

# Continue Training If crushed

In [5]:
import os
import math
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import DataLoader
import numpy as np
import pickle
import gc
from tqdm import tqdm

# Import your custom modules
from dataset import ChessDataset
from model import ChessModel
from helper_funcs import collect_unique_moves, process_data_and_save_chunks

# Set device
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using MPS backend on Apple Silicon (M2).")
else:
    device = torch.device('cpu')
    print("MPS backend not available. Using CPU.")

# Load the move_to_int mapping
with open("../models/mark3_move_to_int.pkl", "rb") as file:
    move_to_int = pickle.load(file)
num_classes = len(move_to_int)

model = ChessModel(num_classes=num_classes).to(device)
model.load_state_dict(torch.load("../models/mark3-continued.pth", map_location=device))

max_games = 300000
chunk_size = 10000

# Assuming data_chunk_files are available
data_chunk_files = [f for f in os.listdir('.') if f.startswith('data_chunk_') and f.endswith('.npz')]
data_chunk_files.sort()

Using MPS backend on Apple Silicon (M2).


/var/folders/84/sbq4d0yx2vdcrx0_5_cy2hmh0000gn/T/ipykernel_10401/2942774120.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../models/

In [6]:
import numpy as np
import os

data_chunk_files = [f for f in os.listdir('.') if f.startswith('data_chunk_') and f.endswith('.npz')]
for file in data_chunk_files:
    try:
        with np.load(file) as data:
            print(f"{file} loaded successfully")
    except EOFError:
        print(f"{file} is corrupted or incomplete.")
        

data_chunk_20.npz loaded successfully
data_chunk_21.npz loaded successfully
data_chunk_23.npz loaded successfully
data_chunk_22.npz loaded successfully
data_chunk_26.npz loaded successfully
data_chunk_27.npz loaded successfully
data_chunk_19.npz loaded successfully
data_chunk_25.npz loaded successfully
data_chunk_24.npz loaded successfully
data_chunk_18.npz loaded successfully
data_chunk_8.npz loaded successfully
data_chunk_9.npz loaded successfully
data_chunk_2.npz loaded successfully
data_chunk_3.npz loaded successfully
data_chunk_1.npz loaded successfully
data_chunk_0.npz loaded successfully
data_chunk_4.npz loaded successfully
data_chunk_5.npz loaded successfully
data_chunk_7.npz loaded successfully
data_chunk_6.npz loaded successfully
data_chunk_15.npz loaded successfully
data_chunk_29.npz loaded successfully
data_chunk_28.npz loaded successfully
data_chunk_14.npz loaded successfully
data_chunk_16.npz loaded successfully
data_chunk_17.npz loaded successfully
data_chunk_13.npz load

In [7]:
additional_epochs = 3  
batch_size = 64  
total_batches_per_epoch = 0
for data_chunk_file in data_chunk_files:
    data = np.load(data_chunk_file)
    num_samples = data['X'].shape[0]
    num_batches = math.ceil(num_samples / batch_size)
    total_batches_per_epoch += num_batches
    data.close()

total_steps = additional_epochs * total_batches_per_epoch
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = OneCycleLR(optimizer, max_lr=0.001, total_steps=total_steps)

# Training loop
for epoch in range(additional_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    total_batches = 0
    for data_chunk_file in tqdm(data_chunk_files, desc=f'Epoch {epoch+1}/{additional_epochs}'):
        # Load data chunk
        data = np.load(data_chunk_file)
        X = torch.tensor(data['X'], dtype=torch.float32)
        y = torch.tensor(data['y'], dtype=torch.long)
        data.close()
        # Create Dataset and DataLoader
        dataset = ChessDataset(X, y)
        dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, shuffle=True)
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
            optimizer.zero_grad()
            outputs = model(inputs)  # Raw logits
            # Compute loss
            loss = criterion(outputs, labels)
            loss.backward()
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()  # Update learning rate
            running_loss += loss.item()
            total_batches += 1
        # Free up memory
        del X, y, dataset, dataloader
        gc.collect()

    end_time = time.time()
    epoch_time = end_time - start_time
    minutes = int(epoch_time // 60)
    seconds = int(epoch_time % 60)
    avg_loss = running_loss / total_batches
    print(f'Epoch {epoch + 1}/{additional_epochs}, Loss: {avg_loss:.4f}, Time: {minutes}m{seconds}s')

Epoch 1/3: 100%|██████████| 30/30 [36:23<00:00, 72.78s/it]


Epoch 1/3, Loss: 1.8970, Time: 36m23s


Epoch 2/3: 100%|██████████| 30/30 [37:12<00:00, 74.41s/it]


Epoch 2/3, Loss: 1.9352, Time: 37m12s


Epoch 3/3: 100%|██████████| 30/30 [35:49<00:00, 71.65s/it]

Epoch 3/3, Loss: 1.8558, Time: 35m49s


In [8]:
# Save the updated model
torch.save(model.state_dict(), "../models/mark3.3-19e.pth")